In [1]:
from dataclasses import dataclass  # python 3.6+
import xarray as xr
from pathlib import Path

In [41]:
#
# What if we have several "cases" that we want to access in different ways
#
# reading:
# - https://www.infoworld.com/article/3563878/how-to-use-python-dataclasses.html
# - @property: https://www.programiz.com/python-programming/property

@dataclass
class CaseDescription:
    '''Object for tracking information about a case/dataset.'''
    name: str
    base_location: Path
    search_str: str=None
    timeseries_location: Path=None
    timeseries_exist: bool = False
    climo_location: Path=None
    climo_exist: bool=False
    @property
    def file_list(self):
        '''makes file_list a class property: ClassDescription.file_list'''
        if self.search_str is not None:
            return sorted(list(self.base_location.glob(self.search_str)))
        else:
            return sorted(list(self.base_location.glob(f"{self.name}*.nc")))
    @property
    def is_multifile(self): 
        '''Tells whether there are multiple files in the file list.'''
        return len(self.file_list) > 1

    def count_files(self):
        if self.file_list is None:
            self.get_base_file_list()
        return len(self.file_list)
    def load_base_dataset(self):
        list_of_datasets = [xr.open_dataset(f) for f in self.file_list]
        return xr.merge(list_of_datasets)
    @staticmethod
    def generate_climatology(ds, v):
        x = ds[v]
        return x.groupby("time.month").mean(dim='time')

In [43]:
tstcase = CaseDescription("B1850_c201_CLOCK", Path("/Users/brianpm/Dropbox/Data/cloud_locking"), search_str="B1850_c201_CLOCK.cam.h0.*")

In [44]:
tstcase

CaseDescription(name='B1850_c201_CLOCK', base_location=PosixPath('/Users/brianpm/Dropbox/Data/cloud_locking'), search_str='B1850_c201_CLOCK.cam.h0.*', timeseries_location=None, timeseries_exist=False, climo_location=None, climo_exist=False)

In [45]:
tstcase.file_list

[PosixPath('/Users/brianpm/Dropbox/Data/cloud_locking/B1850_c201_CLOCK.cam.h0.TS.ncrcat.nc'),
 PosixPath('/Users/brianpm/Dropbox/Data/cloud_locking/B1850_c201_CLOCK.cam.h0.ncrcat.PRECC_PRECL.nc')]

In [46]:
ds = tstcase.load_base_dataset()

In [47]:
climo = tstcase.generate_climatology(ds, "TS")

In [48]:
climo

<xarray.DataArray 'TS' (month: 12, lat: 192, lon: 288)>
array([[[243.95134, 243.95134, 243.95134, ..., 243.95134, 243.95134,
         243.95134],
        [244.70467, 244.67482, 244.55232, ..., 244.73752, 244.72684,
         244.7092 ],
        [245.13547, 245.10855, 245.09381, ..., 245.28998, 245.23836,
         245.19865],
        ...,
        [243.99864, 244.02989, 244.06058, ..., 243.93515, 243.95467,
         243.97388],
        [243.5257 , 243.5377 , 243.55057, ..., 243.48602, 243.4993 ,
         243.51285],
        [243.21198, 243.21307, 243.21402, ..., 243.20804, 243.20944,
         243.21075]],

       [[243.30585, 243.30585, 243.30585, ..., 243.30585, 243.30585,
         243.30585],
        [244.35051, 244.29297, 244.13393, ..., 244.43634, 244.42441,
         244.39008],
        [245.18651, 245.13527, 245.10947, ..., 245.39455, 245.35027,
         245.29031],
...
        [255.62851, 255.6504 , 255.67206, ..., 255.58633, 255.59952,
         255.61223],
        [255.20033, 255.20444, 255.20955, ..., 255.18692, 255.19073,
         255.19562],
        [254.98569, 254.98633, 254.9869 , ..., 254.9834 , 254.98422,
         254.98503]],

       [[231.05887, 231.05887, 231.05887, ..., 231.05887, 231.05887,
         231.05887],
        [231.58139, 231.54494, 231.41644, ..., 231.62323, 231.61371,
         231.59831],
        [232.10411, 232.06143, 232.03899, ..., 232.35948, 232.30023,
         232.21257],
        ...,
        [249.01126, 249.01797, 249.0233 , ..., 248.9828 , 248.99243,
         249.00148],
        [249.11441, 249.11922, 249.12376, ..., 249.10004, 249.10435,
         249.10936],
        [248.98622, 248.98685, 248.98746, ..., 248.98383, 248.98473,
         248.9855 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12